In [1]:
import pandas as pd

In [76]:
df = pd.read_csv('aggregated_hourly.csv')

In [89]:
df["VOC"]

0         464
1         477
2         535
3         566
4         744
         ... 
143833    955
143834    964
143835    983
143836    966
143837    966
Name: VOC, Length: 143838, dtype: int64

In [59]:
df["device_id"].unique()

array(['hka-aqm-a014', 'hka-aqm-a017', 'hka-aqm-a101', 'hka-aqm-a102',
       'hka-aqm-a103', 'hka-aqm-a106', 'hka-aqm-a107', 'hka-aqm-a108',
       'hka-aqm-a111', 'hka-aqm-a112', 'hka-aqm-am001', 'hka-aqm-am002',
       'hka-aqm-am003a', 'hka-aqm-am003b', 'hka-aqm-am004',
       'hka-aqm-am005', 'hka-aqm-am107', 'hka-aqm-am109', 'hka-aqm-am110',
       'hka-aqm-am111', 'hka-aqm-am115', 'hka-aqm-am116', 'hka-aqm-am117',
       'hka-aqm-am123', 'hka-aqm-am124', 'hka-aqm-am126',
       'hka-aqm-am201a', 'hka-aqm-am201b', 'hka-aqm-am204',
       'hka-aqm-am205', 'hka-aqm-am209', 'hka-aqm-am210', 'hka-aqm-am211',
       'hka-aqm-am301', 'hka-aqm-am307', 'hka-aqm-am308'], dtype=object)

In [77]:
a = ["hka-aqm-a017", "hka-aqm-a101", "hka-aqm-a102", "hka-aqm-a103", "hka-aqm-a106", "hka-aqm-a107", "hka-aqm-a108", "hka-aqm-a111", "hka-aqm-a112", "hka-aqm-a014"]

In [80]:
df[~df["device_id"].isin(a)].__len__()

102249

In [85]:
df[df["device_id"].isin(a)].__len__() 

41589

In [73]:
def create_train_data(df, cutoff_date_lower: str, cutoff_Date_upper: str, device_ids : list = [], output_columns : list = ["tmp", "hum", "CO2"]):
    """
    args: 
        df: pandas dataframe
        cutoff_date (str): date to cut off the data
        device_ids (list): list of str, device ids
        output_columns (list): list of str, columns to be output
    
    return:
        pandas dataframe
    """
    return df[(cutoff_Date_upper > df['date_time'] > cutoff_date_lower) & (df["device_id"].isin(device_ids))][output_columns].reset_index(drop=True)

In [72]:
import torch

train = create_train_data(df, "2020-01-01", ["hka-aqm-a017"], ["tmp", "hum", "CO2"])
train = torch.tensor(train.values, dtype=torch.float32)
train.shape

torch.Size([6831, 3])

In [45]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

data_loader = DataLoader(TensorDataset(torch.tensor(train.values).float()), batch_size=32, shuffle=True)

In [53]:
class RNN(nn.Module):

    def __init__(self, input_size= train.shape[1], hidden_size=26, output_size=1, num_layers=1):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out
    
    def train(self, x, y, num_epochs, learning_rate):
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        for epoch in range(num_epochs):
            outputs = self(x)
            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [54]:
RNN_model = RNN(input_size=3, hidden_size=26, output_size=3, num_layers=1)
RNN_model.train(data_loader, 100, 0.01) 

AttributeError: type object 'RNN' has no attribute 'shape'